In [2]:
import pandas as pd
import numpy as np
import scipy as sp
from Data import * 
from Model import * 
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')
import pmdarima as pm
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError
from prophet.diagnostics import cross_validation
from prophet import Prophet
import pmdarima as pm
from pmdarima import pipeline
from pmdarima.pipeline import Pipeline
from pmdarima.preprocessing import BoxCoxEndogTransformer
from sktime.forecasting.tbats import TBATS
from sktime.forecasting.bats import BATS

In [3]:
DAL = get_ETF_data('DAL', '2000-01-01', '2024-01-01')
train_dal, val_dal, test_dal = split_time_series(DAL, 0.6,0.2)

In [4]:
train_dal.index = pd.date_range(start=train_dal.index[0], periods=len(train_dal), freq='B')
val_dal.index = pd.date_range(start=val_dal.index[0], periods=len(val_dal), freq='B')

In [5]:
bats_forecaster = BATS(
    use_box_cox=True,
    box_cox_bounds=(0, 1),
    use_trend=True,
    use_damped_trend=None,
    sp=12,
    use_arma_errors=True,
    show_warnings=True,
    n_jobs=None,
    multiprocessing_start_method='spawn',
    context=None,

)

bats_forecaster.fit(train_dal)


BATS(sp=12, use_box_cox=True, use_trend=True)

In [7]:
fh_list = list(range(1,len(val_dal)+1))

In [8]:
type(fh_list)

list

In [9]:
ypred_bats = bats_forecaster.predict(fh= fh_list)

In [13]:
ypred_bats

,adj_close
2016-12-26,41.785923
2016-12-27,41.767786
2016-12-28,41.661247
2016-12-29,41.666468
2016-12-30,41.693820
...,...
2020-03-06,41.643811
2020-03-09,41.566829
2020-03-10,41.635279
2020-03-11,41.729705


In [16]:
evaluate_forecast(val_dal['adj_close'],ypred_bats['adj_close'])

,MAE,MSE,MAPE (%),sMAPE (%)
0,10.624843,133.993855,24.632423,24.307649
